# Goal
- Image pyramid란 이미지를 다양한 해상도로 표현하는 방법이다.
- cv.pyrUP() 함수를 사용하여 이미지를 확대하고 cv.pyrDown() 함수를 사용하여 이미지를 축소한다.

# Theory
이미지를 다른 해상도로 작업해야 하는 경우 이미지 피라미드를 사용할 수 있다. 이미지 피라미드는 이미지의 다양한 해상도를 나타내는 이미지의 컬렉션이다. 이미지 피라미드는 두 가지 유형이 있다.
1. Gaussian Pyramid
2. Laplacian Pyramid

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

img = cv.imread('Images/messi5.jpg')
assert img is not None, "file could not be read, check with os.path.exists()"
lower_reso = cv.pyrDown(higher_reso)
